## Data Validator
Notebook that reads in old scraped job advertisement data and checks for missing information.

If there is missing informtion, this notebook has the functionality to fill it in.

In [24]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import etree

In [25]:
data_file_path = "../data/2021-12-19-chronicles_of_higher_ed.csv"

In [26]:
csv_data = pd.read_csv(data_file_path, index_col=0)

In [27]:
# This information got a bit mangeld in Cameron's original code, so we'll create a extra 
# variable to hold these strings in case they change in the future.
date_posted_str= "Date Posted"
job_url_str = "Job URL"

In [43]:
def get_urls_missing_date_posted(df):
    r"""Gets urls for jobs with missing date posted data.

    Reads in DataFrame and looks for missing date posted information, then returns a 
    DataFrame with all the urls to the jobs with missing date posted information.

    Parameters
    ----------
    df : DataFrame
        DataFrame containing a column of data with date posted information and a column
        of information with original request url information.
    
    Returns
    -------
    Series
        Pandas Series (with ID) of urls to pages where posted date was missed.
    """
    no_dates = df[df[date_posted_str].isna()]
    urls = no_dates.loc[:,job_url_str]
    return urls


In [47]:
def get_page_from_url(url):
    r"""Makes request for page.


    Parameters
    ----------
    url : str
        Url of page to request.

    Returns
    -------
    requests.models.Response
        A request Response object.

        Access the page content using .content.
    """
    return requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

def get_page_content_bytes(url):
    r"""Gets page content from a url
    
    Parameters
    ----------
    url : str
        Url of page to get content of.

    Returns
    -------
    bytes
        Page content of Response object in bytes. 

        Parsing the bytes of the page content is suppored by BeautifulSoup.
    """
    page = get_page_from_url(url)
    return page.content
    

In [48]:
def get_date_posted(page_content):
    soup = BeautifulSoup(page_content, "html.parser")
    dom = etree.HTML(str(soup))
    date_xpath = '//*[@id="main"]/div/div[3]/div[1]/div[1]/div[1]/div/div[1]/dl/dd[4]'
    date_element = dom.xpath(date_xpath)
    return date_element[0].text    

In [49]:
url = "https://jobs.chronicle.com/job/37302598/assistant-professor-in-chemistry-open-area-/"
print(type(get_page_content_bytes(url)))

<class 'bytes'>


In [ ]:
# pd.to_datetime(listing_df["Posted Date"],infer_datetime_format=True)